In [1]:
from __future__ import division, print_function
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import csv
import scikitplot as skplt
from sklearn import preprocessing

from pylab import scatter,show,plot
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

from sklearn.metrics import precision_score,recall_score,accuracy_score

In [2]:
df = pd.read_csv('deliveries.csv')


In [3]:
df.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [4]:
# Converting type of columns to category 
df['batting_team'] = df['batting_team'].astype('category') 
df['bowling_team'] = df['bowling_team'].astype('category') 
df['batsman'] = df['batsman'].astype('category')
df['non_striker'] = df['non_striker'].astype('category')
df['bowler'] = df['bowler'].astype('category')


# Assigning numerical values and storing it in another columns 
#df['Season'] = df['Season'].cat.codes 
df['batting_team'] = df['batting_team'].cat.codes 
df['bowling_team'] = df['bowling_team'].cat.codes 
df['batsman'] = df['batsman'].cat.codes 
df['non_striker'] = df['non_striker'].cat.codes 
df['bowler'] = df['bowler'].cat.codes 


In [11]:
df.head()

df.info()

df.drop(['player_dismissed','dismissal_kind','fielder'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179078 entries, 0 to 179077
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          179078 non-null  int64 
 1   inning            179078 non-null  int64 
 2   batting_team      179078 non-null  int8  
 3   bowling_team      179078 non-null  int8  
 4   over              179078 non-null  int64 
 5   ball              179078 non-null  int64 
 6   batsman           179078 non-null  int16 
 7   non_striker       179078 non-null  int16 
 8   bowler            179078 non-null  int16 
 9   is_super_over     179078 non-null  int64 
 10  wide_runs         179078 non-null  int64 
 11  bye_runs          179078 non-null  int64 
 12  legbye_runs       179078 non-null  int64 
 13  noball_runs       179078 non-null  int64 
 14  penalty_runs      179078 non-null  int64 
 15  batsman_runs      179078 non-null  int64 
 16  extra_runs        179078 non-null  int

In [13]:
print('KMeans Labelling')
n_clusters=3
kmeans = KMeans(n_clusters).fit(df)
class_labels = kmeans.labels_

KMeans Labelling


C:\Users\saura\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [17]:

X_train, X_test, y_train, y_test = train_test_split(df, class_labels, test_size=0.30, random_state=42)
#gnb.fit(X_train,np.ravel(y_train))
#print(y_train[1])
#data = np.asarray(data)
#data = data.tolist()
X_train = np.asarray(X_train)
X_train = X_train.tolist()
tr_sam = len(X_train)

y_train = np.asarray(y_train)
y_train = y_train.tolist()

classes = []

if len(classes) == 0:
    for i in range(0,n_clusters):
        classes.append([])

for i in range(0,len(y_train)):
    if y_train[i] == 0:
        classes[0].append(X_train[i])
    elif y_train[i] == 1:
        classes[1].append(X_train[i])
    elif y_train[i] == 2:
        classes[2].append(X_train[i])

print("class 0", classes[0])

prob_class0 = len(classes[0]) / tr_sam
print("P(Class = 0) is ",prob_class0)

print("*****************************************************************")
print("class 1")
print(classes[1])

prob_class1 = len(classes[1]) / tr_sam
print("P(Class = 1) is ",prob_class1)

print("*****************************************************************")
print("class 2")
print(classes[2])

prob_class2 = len(classes[2]) / tr_sam
print("P(Class = 2) is ",prob_class2)
print("*****************************************************************")



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:

#print(len(classes[0][0]))
def freq(buff):
    zeros = 0
    ones = 0
    for i in range(0,len(buff)):
        if buff[i] == 0:
            zeros = zeros+1
        elif buff[i] == 1:
            ones = ones+1
    return zeros,ones

prob_mat1 = [[]]

for k in range(0,len(classes[0][0])):
    buff = []
    for j in range(0,len(classes[0])):
        buff.append(classes[0][j][k])
    #print(buff)
    #print(np.asarray(buff))
    buff = np.asarray(buff)
    buff = np.reshape(buff,(-1,1))
    #print(len(buff))
    zeros,ones = freq(buff)
    prob_zeros_class0 = zeros/len(classes[0])
    prob_ones_class0 = ones/len(classes[0])
    if prob_mat1[0] == []:
        prob_mat1[0].append(prob_zeros_class0)
        prob_mat1[0].append(prob_ones_class0)
    else:
        l = len(prob_mat1)
    prob_mat1.append([])
    nl = l+1
    prob_mat1[nl-1].append(prob_zeros_class0)
    prob_mat1[nl-1].append(prob_ones_class0)

print("Probabilities of Class 0",prob_mat1)

prob_mat2 = [[]]

for k in range(0,len(classes[1][0])):
    buff = []
    for j in range(0,len(classes[1])):
        buff.append(classes[1][j][k])

    buff = np.asarray(buff)
    buff = np.reshape(buff,(-1,1))
    
    zeros,ones = freq(buff)
    prob_zeros_class1 = zeros/len(classes[1])
    prob_ones_class1 = ones/len(classes[1])
    if prob_mat2[0] == []:
        prob_mat2[0].append(prob_zeros_class1)
        prob_mat2[0].append(prob_ones_class1)
    else:
        l = len(prob_mat2)
prob_mat2.append([])
nl = l+1
prob_mat2[nl-1].append(prob_zeros_class1)
prob_mat2[nl-1].append(prob_ones_class1)

print("\nProbabilities of Class 1",prob_mat2)

prob_mat3 = [[]]

for k in range(0,len(classes[2][0])):
    buff = []
    for j in range(0,len(classes[2])):
        buff.append(classes[2][j][k])

    buff = np.asarray(buff)
    buff = np.reshape(buff,(-1,1))
    
    zeros,ones = freq(buff)
    prob_zeros_class2 = zeros/len(classes[2])
    prob_ones_class2 = ones/len(classes[2])
    if prob_mat3[0] == []:
        prob_mat3[0].append(prob_zeros_class2)
        prob_mat3[0].append(prob_ones_class2)
    else:
        l = len(prob_mat3)
prob_mat3.append([])
nl = l+1
prob_mat3[nl-1].append(prob_zeros_class2)
prob_mat3[nl-1].append(prob_ones_class2)

print("\nProbabilities of Class 2",prob_mat3)


print("\n Testing")
print("\n New Sample: 0 0 0 0 0 0 0 0 1 0 0 0 1 0")

samp = [ 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]

probofsam_class0 = 1
j = 0
for i in range(0,len(prob_mat1)):
    probofsam_class0 = prob_mat1[i][samp[j]] * probofsam_class0
    j = j+1

print("\nProbability of sample belonging to class: 0 is ",probofsam_class0)

probofsam_class1 = 1
j = 0
for i in range(0,len(prob_mat2)):
    probofsam_class1 = prob_mat2[i][samp[j]] * probofsam_class1
    j = j+1

print("\nProbability of sample belonging to class: 1 is ",probofsam_class1)

probofsam_class2 = 1
j = 0
for i in range(0,len(prob_mat3)):
    probofsam_class2 = prob_mat3[i][samp[j]] * probofsam_class2
    j = j+1

print("\nProbability of sample belonging to class: 2 is ",probofsam_class2)

ch = max(probofsam_class0,probofsam_class1,probofsam_class2)
if ch==probofsam_class0:
    print("\n The given sample belongs to class: ",0)
elif ch==probofsam_class1:
    print("\n The given sample belongs to class: ",1)
elif ch==probofsam_class2:
    print("\n The given sample belongs to class: ",2)

Probabilities of Class 0 [[0.0, 0.0016612714897333422], [0.0, 0.0016612714897333422], [0.0, 0.5195412992092348], [0.1043563284950779, 0.05980577363040032], [0.10428987763548855, 0.06004309812893365], [0.0, 0.053293589390645615], [0.0, 0.16135218006284352], [0.001310031231904007, 6.645085958933369e-05], [0.0008543681947200045, 8.543681947200046e-05], [0.00187961002838401, 0.001509383810672008], [0.9994494071634027, 0.0005505928365973363], [0.9696224641877331, 0.027453697990336146], [0.9972470358170133, 0.00187961002838401], [0.9823905222088266, 0.015407106444784081], [0.9959085256452853, 0.004015530515184021], [0.9999905070200586, 0.0], [0.40624258360942084, 0.36856494622226865], [0.9451685478588584, 0.04875594497868826], [0.3536609677143752, 0.41373254478313287]]

Probabilities of Class 1 [[0.0, 0.0], [0.3493253373313343, 0.3737131434282859]]

Probabilities of Class 2 [[0.0, 0.0], [0.3337330135891287, 0.3771982414068745]]

 Testing

 New Sample: 0 0 0 0 0 0 0 0 1 0 0 0 1 0


IndexError: list index out of range